# **LINEARISATION UNIQUE DES TRAFICS SUR BORDEAUX METROPOLE**
> ATTENTION : IL Y A DES MELI-MELO ENTRE LE FICHIER gdf_rhv_groupe qui contient un idtronc pour tout les troncons et a été constitué avant de prendre en compte les cat_rhv 62, 63 etc...; qui est utilisé dans toute la partie affectation points de comptage, et le fichier issus de la Bdd graph_filaire qui lui contient bien toute les lignes mais n'a pas d'idtronc renseigné.<br> il faudrait remettre tout ça à plat

In [1]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\marti\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\marti\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\marti\git\Lin_uniq_BdxM\Lin_uniq_BdxM\Traitements')
import Connexion_Transfert as ct
import geopandas as gp
import pandas as pd
import numpy as np
from shapely.geometry import MultiPoint, Polygon, box
from shapely.ops import nearest_points
from shapely.wkt import dumps, loads
import re, json
from copy import copy, deepcopy
from difflib import get_close_matches,SequenceMatcher
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from collections import Counter
from Outils import creer_graph, plus_proche_voisin,nb_noeud_unique_troncon_continu,verif_index, gp_changer_nom_geom

from Simplifier_Rdpt import creer_dico_noeud_rdpt,simplifier_noeud_rdpt,maj_graph_rdpt, donnees_tot_rd_pt
import Estim_trafic as et
import Comptage as Cpt
import Affectation_pt_comptage as ap
import Voies_Cat4 as v4
from MMM import import_fichiers_mmm

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 50)


from Base_BdTopo.Import_outils import import_donnes_base
from Base_BdTopo.Rond_points import identifier_rd_pt
from Base_BdTopo.Regroupement_correspondance import *
from Base_BdTopo.Troncon_elementaire import *
from Base_BdTopo.Troncon_base import *
from Base_BdTopo.Gestion_2_chaussee import *
import plotly.express as px


## 1 Regrouper les troncons du filaire de voie
>Les troncons du filaire de voie sont regroupés d'abord uniquement avec les troncons de cat_rhv égal à 1,2,3,61,62,63, pour pouvoir réaliser des troncon de trafic homogènes tous reliés entre eux.
<br> Ensuite, un deuxième regroupement comprenant l'ensemble des troncons de cat_rhv permetd'affiner le diagnostic

> ### 1.1 Import du fichier final
le fichier shape est vérifié à la main car l'affectation automatique n'est pas 100% fiable

In [4]:
# Pour info : lire le graph issu de la methode decrite dessous
with ct.ConnexionBdd('local_otv') as c :
    graph_filaire_123 = gp.read_postgis('select * from linearisation_bm.graph_rhv_123_v2', c.connexionPsy)
    graph_filaire_123_vertex = gp.read_postgis('select id,cnt,chk,ein,eout,the_geom as geom from linearisation_bm.graph_rhv_123_v2_vertices_pgr', c.connexionPsy)
    graph_filaire = gp.read_postgis('select * from linearisation_bm.graph_rhv_complet', c.connexionPsy)
    graph_filaire_vertex=gp.read_postgis('select id,cnt,chk,ein,eout,the_geom as geom from linearisation_bm.graph_rhv_complet_vertices_pgr', c.connexionPsy)

In [5]:
gdf_rhv_groupe_123=graph_filaire_123
"""gdf_rhv_groupe_123=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_groupe_123_v2.shp').merge(
    graph_filaire_123[['ident','source','target']], on='ident', how='left')"""
gdf_rhv_groupe=gp.read_file(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\rhv_grp.shp').merge(
    graph_filaire[['ident','source','target']], on='ident', how='left')

>### 1.2 Méthode
 pour information

In [ ]:
# 1. import du fichier de filaire de base
gdf_rhv=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\Filaire_voie\FV_TRONC_L.shp')
# 2. mise en forme
gdf_rhv.columns=[a.lower() for a in gdf_rhv.columns] #nom de colonne en minuscule
gdf_rhv.rename(columns={'gid':'id','nom_voie' : 'numero'},inplace=True)
gdf_rhv=gdf_rhv.loc[~gdf_rhv.id.isna()].copy()
gdf_rhv['id_ign']=gdf_rhv.ident.apply(lambda x : 'TRONROUT'+str(x))
gdf_rhv=gdf_rhv.loc[~gdf_rhv.cat_rhv.isin(['98','99'])].copy()
gdf_rhv['nature']=gdf_rhv.apply(lambda x : 'Route à 2 chaussées' if x['rgraph_dbl']==0 else 'Route à 1 chaussée', axis=1)
gdf_rhv['nature']=gdf_rhv.apply(lambda x : 'Bretelle' if x['cat_dig']=='8' else x['nature'], axis=1)
gdf_rhv['numero']=gdf_rhv.apply(lambda x : 'Bretelle '+x['numero'] if x['cat_dig']=='8' else x['numero'], axis=1)
gdf_rhv['sens']=gdf_rhv.apply(lambda x : 'Direct' if x['rgraph_dbl']==0 else 'Double', axis=1)
gdf_rhv['codevoie_d']='NR'
gdf_rhv['importance']=gdf_rhv['cat_dig']
gdf_rhv['id']=gdf_rhv.id.apply(lambda x : int(x))
#gdf_rhv.loc[gdf_rhv.nom_voie.isna()] #verif des nom_voie null : 2 lignes, cf traitements plus loins
#filtrer les voies cylcables et autres
gdf_rhv_filtre=gdf_rhv.loc[~gdf_rhv.cat_dig.isin(['6','7','9','10'])].copy()

# 3. creer le graph en bdd
#creer_graph(gdf_rhv_filtre.loc[gdf_rhv_filtre.cat_rhv.isin(['1','2','3','33','61','62','63'])],
            #'local_otv',schema='linearisation_bm',table='graph_rhv_123',table_vertex='graph_rhv_123_vertices_pgr')
creer_graph(gdf_rhv_filtre,'local_otv',schema='linearisation_bm',table='graph_rhv_complet',table_vertex='graph_rhv_complet_vertices_pgr')

In [ ]:
# 4. initialiser les paramètre de regroupement
df=import_donnes_base('local_otv','linearisation_bm', 'graph_rhv_complet','graph_rhv_complet_vertices_pgr')
df2_chaussees=df.loc[df.nature.isin(['Autoroute', 'Quasi-autoroute', 'Route à 2 chaussées'])]
df_avec_rd_pt,carac_rd_pt,lign_entrant_rdpt=identifier_rd_pt(df)
df_lignes=df_avec_rd_pt.set_index('id_ign')#mettre l'id_ign en index
bretelle=df_avec_rd_pt.loc[df_avec_rd_pt['nature']=='Bretelle'].copy()
bretelle_tri=bretelle.loc[bretelle.length.sort_values(ascending=False).index.tolist()].id_ign.tolist()
sans_bretelle=df_avec_rd_pt.loc[df_avec_rd_pt['nature']!='Bretelle'].copy()
list_sans_bretelle=sans_bretelle.id_ign.tolist()
list_tri_longueur=sans_bretelle.loc[sans_bretelle.length.sort_values(ascending=False).index.tolist()].id_ign.tolist()
list_id_ign=bretelle_tri+list_tri_longueur

In [ ]:
# 5. appel fonction de grouepement pour creer des idtronc pour uniquement voies de 1 à 3 et d'un autre coté pour toute les voies
df_affectation, dico_erreur, lignes_traitees, lignes_non_traitees=regrouper_troncon(
    list_id_ign, df_avec_rd_pt, carac_rd_pt,df2_chaussees,[])

# 6. mise en forme et export
gdf_rhv_groupe=gdf_rhv_filtre.merge(df_affectation, left_on='id_ign', right_on='id', how='left')
gdf_rhv_groupe.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\rhv_grp_v2.shp')
#gdf_rhv_groupe_123=gdf_rhv_filtre.merge(df_affectation, left_on='id_ign', right_on='id', how='left')
#gdf_rhv_groupe_123.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_groupe_123.shp')

## 1. MISE EN FORME DES COMPTAGES PONCTUELS
Pour ces comptages l'idée c'est de réussir à regrouper les comptages par un idtronc issu de la detection des troncon elementaire. ça a du sens car pour un comptage ponctuel normalement les 2 sens de cicru sont sur un troncon unique. DE plus, les ens circu, noms de voie et autres attributs permettant de regrouper les comptages sont de qualité trop variable pour s'appuyer uniquement dessus. et enfin, peu de 2*2 vois sont meusrés

In [6]:
affect_finale,cpt_pct_l93=Cpt.traitements_bdxm_pct(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\comptages\base_comptages_routiers_finale.xlsx',gdf_rhv_groupe)

## 2. MISE EN FORME DES COMPTAGES PERMANENTS
Pour ces compteurs on va regrouper en séparant les siredo des boucles liées au système gertrude. Pour les siredo pas de pb, pour les autre sc'est plus compliqué car les références de localiusation varient parfois. Il y a donc une phase de regroupement manuel à la fin

> ### 2.1 Import du fichier final

In [7]:
#import fichier corrige_final
cpt_perm_final=gp.read_file(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\cpt_perm_groupe.shp')

> ### 2.1 Méthode
Pour info

In [ ]:
#creer un gdf avec la fusion des points et des données de comptages non nulles
cpt_brut=pd.read_excel(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\comptages\comptage_trafic_final.xls')
cpt_brut.columns=[a.lower() for a in cpt_brut.columns]
cpt_brut=cpt_brut.loc[~cpt_brut.mjo_val.isna()].copy()
pt_cpt_perm=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\comptages\PC_CAPTE_P.shp')
pt_cpt_perm.columns=[a.lower() for a in pt_cpt_perm.columns]
pt_cpt_perm=pt_cpt_perm.merge(cpt_brut, on='ident')
pt_cpt_perm['nom_voie']=pt_cpt_perm.nom_voie.apply(lambda x : re.sub(('é|è|ê'),'e',x.lower().replace('  ',' ')))#mise en forme nom de voie

#regrouper les comptages
#un compteur peut caracteriser 1 ligne, ou 2 compteur 1 ligne ou 2 compteurs 2 lignes, dc les compteurs ne vont pas toujours par deux

#regrouper geographiquement
pt_cpt_perm['x_l93']=pt_cpt_perm.geometry.apply(lambda x : x.x)
pt_cpt_perm['y_l93']=pt_cpt_perm.geometry.apply(lambda x : x.y)
data_test_clust=[[x, y] for x, y in zip(pt_cpt_perm.x_l93.tolist(), pt_cpt_perm.y_l93.tolist())]
db = DBSCAN(eps=200, min_samples=2).fit(data_test_clust)
labels = db.labels_
pt_cpt_perm['n_cluster']=labels

#correspondance nom_rue
#analyser les noms de commune : 
#grp_nom_voie.nom_voie.apply(lambda x : x.split()[0]).unique()#liste des permiers mots
#grp_nom_voie.loc[grp_nom_voie.nom_voie.apply(lambda x : x.split()[0]=='carbon')].nom_voie.unique() #test des valeusr
dico_commune={'le bouscat','le haillan','bordeaux','pessac','talence', 'merignac','st medard','carbon blanc',
       'begles', 'eysines','villenave d\'ornon','bruges','gradignan','bassens','cenon','floirac','pessa','lormont', 'artigues'}
dico_type_voie={'allee' : ['all.','allee', 'allees'],'avenue': ['av', 'av.', 'avenue'], 'boulevard' : ['blvd'],
               'bretelle' : 'bretelle', 'cours' : ['crs','cours'], 'cote' : ['côte'], 'place':['pl'], 'route':['route', 'rte'],'rue':['rue'],
               'voie':['voie']}
def decoupe_nom_voie(nom_voie) : 
    for c in dico_commune : 
        if nom_voie[:len(c)]==c :
            commune=nom_voie[:len(c)]
            nom_voie=nom_voie[len(c):].strip()
    for k, v in dico_type_voie.items() : 
        if any([a for a in nom_voie.split() if a in v])  : 
            caractere=[a for a in nom_voie.split() if a in v][0]
            type_voie=k
            nom_voie=' '.join([n for n in nom_voie.split() if n!=caractere])
            break
    else : type_voie=None
    for test in ['avant','apres'] :
        if test in nom_voie : 
            return nom_voie.split(test)[0].strip(), test,nom_voie.split(test)[1].strip(), commune, type_voie
    else : return nom_voie, nom_voie,nom_voie,commune,type_voie

pt_cpt_perm['nom_rue']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[0])
pt_cpt_perm['localisant']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[1])
pt_cpt_perm['rue_refer']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[2])
pt_cpt_perm['commune']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[3])
pt_cpt_perm['type_voie']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[4])
pt_cpt_perm.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\pt_cpt_perm.shp')

>>#### 2.2.1 SIREDO

In [ ]:
siredo=pt_cpt_perm.loc[pt_cpt_perm['type']=='SIREDO'].copy()#isoler siredo
cross_join=siredo.assign(key=1).merge(siredo.assign(key=1), on='key').drop('key',axis=1)#calcul des distances
cross_join['distance']=cross_join.apply(lambda x : x['geometry_x'].distance(x['geometry_y']),axis=1)
cross_join_filtre=cross_join[[c for c in cross_join.columns if c[-2:]=='_x']+['distance','ident_y']].rename(columns=
              {c : c[:-2] for c in cross_join.columns if c[-2:]=='_x'}).copy()#filtre des attribut
cross_join_filtre=cross_join_filtre.loc[(cross_join_filtre['ident']!=cross_join_filtre['ident_y'])].copy()
siredo_proches=cross_join_filtre.loc[(cross_join_filtre.groupby('ident')['distance'].transform(min)==
                                      cross_join_filtre['distance'])].copy()#plus proche voisin
siredo_proches['id_grpsiredo']=siredo_proches.reset_index().index
id_siredo_proches=(pd.concat([siredo_proches[['ident','ident_y', 'id_grpsiredo']],
           siredo_proches[['ident_y','ident', 'id_grpsiredo']].rename(columns={
    'ident_y':'ident', 'ident':'ident_y'})],sort=False).sort_values('ident').
    reset_index().drop_duplicates('index').drop_duplicates(['ident','ident_y']).drop('index',axis=1))
cle_assoc_siredo=pd.concat([id_siredo_proches[['ident','id_grpsiredo']],id_siredo_proches[['ident_y','id_grpsiredo']].rename(columns={
    'ident_y':'ident'})]).sort_values('id_grpsiredo')

>>#### 2.2.2 Gertrude
> au final ce n'est pas très efficace, et un renseignement des regroupement a la ain aurait été plus rapide. il sera important si de nouvelles stations sont posées de fournir de suite un identifiant de regroupement

In [ ]:
gertrude=pt_cpt_perm.loc[pt_cpt_perm['type']=='BOUCLE'].copy()#isoler les boucles
gertrude['x']=gertrude.geometry.apply(lambda x : x.x)
gertrude['y']=gertrude.geometry.apply(lambda x : x.y)

In [ ]:
#si c'est sur le mm idtronc on regroupe

#rappatrier le numero d'id_tronc pour chaque comptage
#trouver la distance min à chaque objet ligne"du rhv groupe
grp_troncon_temp=gertrude.copy()
grp_troncon_temp.geometry=grp_troncon_temp.buffer(20)#passer la geom en buffer
intersct_buff_20=gp.sjoin(grp_troncon_temp,gdf_rhv_groupe,how='left',op='intersects')
intersct_buff_20.geometry=gertrude.geometry#repasser la geom en point
intersct_buff_20=intersct_buff_20.merge(gdf_rhv_groupe[['ident','geometry']], left_on='ident_right', right_on='ident')
intersct_buff_20['dist_pt_ligne']=intersct_buff_20.apply(lambda x : x['geometry_x'].distance(x['geometry_y']), axis=1) #définir la disance entre les points et ligne
joint_dist_min=intersct_buff_20.loc[intersct_buff_20.groupby('ident_left')['dist_pt_ligne'].transform(min)==intersct_buff_20['dist_pt_ligne']][['ident_left','idtronc','numero']].rename(
    columns={'ident_left':'ident'}).copy()#ne garder que la ligne la plus proche
grp_troncon=gertrude.merge(joint_dist_min, on='ident',how='left')#df finale

#regrouper
grp_idtronc=grp_troncon.groupby('idtronc')['ident'].agg(lambda x : tuple(x))
grp_idtronc=grp_idtronc.loc[grp_idtronc.apply(lambda x : len(x)>1)].reset_index().copy()
grp_idtronc['id_grp']=grp_idtronc.ident.rank()
#mettre en forme
grp_idtronc=pd.DataFrame([(a,g) for i,g in zip(grp_idtronc.ident.tolist(),grp_idtronc.id_grp.tolist()) for a in i], columns=['ident','id_grp'])
grp_final=grp_idtronc.copy()

In [ ]:
# trouver les points qui restent et préparer un regrouepement par commune, nom de rue, distance et se,s

#points qui restent
gertrude_etape2=gertrude.loc[~gertrude['ident'].isin(grp_idtronc.ident.tolist())].copy()
#comparer les noms
cross_join=gertrude_etape2[['gid','geometry','ident','nom_voie', 'sens_cir','commune', 'nom_rue', 'localisant','rue_refer','type_voie']].assign(key=1).merge(
    gertrude_etape2[['gid','geometry','ident','nom_voie', 'sens_cir','commune', 'nom_rue', 'localisant','rue_refer','type_voie']].assign(key=1), on='key').drop('key',axis=1)#calcul des distances
cross_join['distance']=cross_join.apply(lambda x : x['geometry_x'].distance(x['geometry_y']),axis=1)
cross_join['comp_nom_rue']=cross_join.apply(lambda x : SequenceMatcher(None,x['nom_rue_x'], x['nom_rue_y']).ratio(), axis=1)#affecter une note a cahque relation

#vérifier que les sens de cicru sont cohérents et ne garder que les lignes avec un sens ok, un nom de voie proche et une distance proche
gertrude_join=cross_join.loc[(cross_join['comp_nom_rue']>0.6) & (cross_join['distance']<1000) & (cross_join['commune_x']==cross_join['commune_y']) & 
                             (cross_join['type_voie_x']==cross_join['type_voie_y'])].copy()
#on ajoute un attribut de verif du sens
def verif_sens(s1,s2) : 
    for sens in [['Sens Sortant', 'Sens Entrant'],['Sens N > S', 'Sens S > N'], ['Sens Nord > Sud', 'Sens Sud > Nord'],['Nord > Sud', 'Sud > Nord'],
                ['Boulevards Nord > Sud', 'Boulevards Sud > Nord']] :
        if s1 in sens and s2 in sens : 
            if s1!=s2 : 
                return True
        else : 
            continue
    return False
gertrude_join['valid_sens']=gertrude_join.apply(lambda x : verif_sens(x['sens_cir_x'],x['sens_cir_y']),axis=1)  
gertrude_join['id_grp']=gertrude_join.sort_values('ident_x').ident_x.rank(method="min")
gertrude_grp=gertrude_join.loc[gertrude_join['valid_sens']][['ident_x','ident_y','distance', 'comp_nom_rue', 'valid_sens','id_grp']].sort_values('ident_x')

In [ ]:
#obtenir une liste des ident qui se regroupent
list_id_grp=pd.concat([gertrude_grp[['ident_x','id_grp']],gertrude_grp[['ident_y','id_grp']].rename(columns={'ident_y' : 'ident_x'})]).drop_duplicates([
    'ident_x','id_grp']).groupby('id_grp').ident_x.agg(lambda x : set(sorted(tuple(x)))).tolist()
dico={}
for e,c in enumerate(gertrude_grp.ident_x.tolist()) : 
    for l in list_id_grp : 
        if c in l :
            #print('c dans l','c : ',c,'l :',l)
            if c not in dico.keys() :
                dico[c]=list(l)
            else : 
                dico[c]+=list(l)
list_id_grp_temp=set([ v for v in {k : tuple(set(sorted(dico[k]))) for k in dico.keys()}.values()])
list_id_grp_final=set(v for v in {c :b for c in gertrude_grp.ident_x.tolist() 
 for b in [a for a in list_id_grp_temp if c in a] 
 if len(b) == max([len(c) for c in [a for a in list_id_grp_temp if c in a]])}.values())

In [ ]:
#affecter un identifiant
def groupe_pt_gertrude_etape2(df_local) : 
    """
    regrouper les points à partir d'une liste d'ident selon l'ordre d'apparition 
    """
    limite_box=MultiPoint([(a.x,a.y) for a in df_local.itertuples()]).bounds
    plg_long, plg_larg=limite_box[2]-limite_box[0],limite_box[3]-limite_box[1]
    if max(plg_long,plg_larg)==plg_long : 
        df_local['next_sens']=df_local.sort_values('x').sens_cir.shift(-1,fill_value='NC')
        df_local['prev_sens']=df_local.sort_values('x').sens_cir.shift(1,fill_value='NC')
    else : 
        df_local['next_sens']=df_local.sort_values('y').sens_cir.shift(-1,fill_value='NC')
        df_local['prev_sens']=df_local.sort_values('y').sens_cir.shift(1,fill_value='NC')
    df_local['sens_comp']=df_local.apply(lambda x : x.prev_sens if x.next_sens=='NC' else x.next_sens,axis=1)
    df_local['verif_sens']=df_local.apply(lambda x : verif_sens(x['sens_cir'],x['sens_comp']),axis=1)
    df_local['id_grp']=-99
    i=0
    while i < len(df_local)-1:
        if df_local.iloc[i].verif_sens : 
            df_local.loc[df_local.index.isin(df_local.iloc[i:i+2].index.tolist()),'id_grp']=i
            i+=2
        else : 
            i+=1
    return

# à partior de la liste des ident groupé : 
grp_final2=grp_final.copy()
# on selectionne ces points dans la df
increment=grp_final2.id_grp.max()
for l in list_id_grp_final : 
    #if 'Z8CT13' not in l : continue
    df_local=gertrude_etape2.loc[gertrude_etape2.ident.isin(l)].copy()
    if len(df_local)==1:
        continue
    if len(df_local)==2 : 
        if (gertrude_join.loc[(gertrude_join.ident_x.isin(l)) & (gertrude_join['ident_x'] != gertrude_join['ident_y'])].valid_sens).all() : 
            increment+=1
            df_local['id_grp']=increment
            grp_final2=pd.concat([grp_final2,df_local[['ident','id_grp']]],sort=False)
    else :
        groupe_pt_gertrude_etape2(df_local)
        increment+=grp_final2.id_grp.max()
        df_local.loc[df_local['id_grp']!=-99,'id_grp']=df_local['id_grp']+increment
        grp_final2=pd.concat([df_local[['ident','id_grp']],grp_final2],sort=False)

In [ ]:
#rappatrier l'id_grp sur les données de base et export pour modif manuelle
gertrude.merge(grp_final2, on='ident',how='left').to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gertrude.shp')

#import des modifs manuelles et mise en forme finale
gertrude_manu=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gertrude.shp')
gertrude_manu.loc[(gertrude_manu.id_grp.isna()) | (gertrude_manu.id_grp==-99),'id_grp']=gertrude_manu.loc[(gertrude_manu.id_grp.isna()) | (gertrude_manu.id_grp==-99),'id_grp'].reset_index().index + gertrude_manu.id_grp.max()
gertrude_manu.loc[gertrude_manu.type_grp.isnull(),'type_grp']='auto'
#export pour dernière modif manuelle
gertrude_manu.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gertrude_final.shp')

#fichier final
gertrude_fin=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gertrude_final.shp')
#concat avec les siredo
cle_assoc_siredo['id_grpsiredo']=cle_assoc_siredo['id_grpsiredo']+gertrude_fin.id_grp.max()
cpt_perm_final=pd.concat([gertrude_fin,siredo.merge(cle_assoc_siredo, on='ident').rename(columns={'id_grpsiredo':'id_grp'})],axis=0, sort=False)
cpt_perm_final.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\cpt_perm_groupe.shp')

# 3. Affecter du trafic au filaire de voie

> ### On commence par affecter le trafic aux troncons des catégories 1,2,3 à partirdes comptages permanents.<br> Pour le détail des fonctions cf module Affectation_pt_comptage

In [155]:
gdf_rhv_cpt_perm_123,lgn_proche_perm=ap.affectation_cpt_perm(cpt_perm_final,gdf_rhv_groupe,gdf_rhv_groupe_123,10)
#ap.export_cpt_perm_linearises(gdf_rhv_cpt_perm_123,r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_123_cpt_perm.shp')

> ### ensuite les comptages ponctuels

In [ ]:
#affectation des comptages ponctuels et exports
gdf_traf_tot=ap.affectation_cpt_ponct(cpt_pct_l93, affect_finale, lgn_proche_perm, gdf_rhv_groupe, 10, gdf_rhv_groupe_123, gdf_rhv_cpt_perm_123)
#ap.export_cpt_ponct_linearises(gdf_traf_tot,r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_123_cpt_perm-ponct.shp')

## 2. MMM
En premier lieu il faut mettre à jour les attributs en renomant de façon explicite puis en faisant la somme des 2 sens de circulation pour les voies doucble sens cf module MMM

In [ ]:
fichier_src_simpl=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\MMM\MMM_Maj_EDA2017\simplifie\MMM_simple_2019.shp')

In [ ]:
#creer graph et importer 
creer_graph(fichier_src_simpl, 'local_otv',id_name='NO', schema='linearisation_bm', table='mmm_graph_2019', table_vertex='mmm_graph_2019_vertices_pgr')

In [ ]:
#importer le graph
with ct.ConnexionBdd('local_otv') as c :
    graph_mmm_filaire = gp.read_postgis('select * from linearisation_bm.mmm_graph_2019', c.connexionPsy)
    graph_mmm_filaire_vertex = gp.read_postgis('select * from linearisation_bm.mmm_graph_2019_vertices_pgr', c.connexionPsy,geom_col='the_geom')

In [ ]:
#importer les fichiers finaux
mmm_simple,cle_mmm_rhv=import_fichiers_mmm(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\MMM\MMM_Maj_EDA2017\simplifie\MMM_simple_2019.shp',
                                            r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\appariementV0.shp')

# Affecter du trafic aux voies non connues

In [ ]:
# METHODE A UTILISER
gdf_base=gdf_traf_tot.copy()
lgn_rdpt, dico_noeud, gdf_rhv_rdpt_simple=donnees_tot_rd_pt(gdf_base, 'local_otv','linearisation_bm', 'graph_rhv_123_v2','graph_rhv_123_v2_vertices_pgr')
gdf_rhv_rdpt_simple5=gdf_rhv_rdpt_simple.copy()

et.calculer_trafic_3voies(gdf_base,gdf_rhv_rdpt_simple5,lgn_rdpt,graph_filaire_123_vertex)

gdf_calcul=gdf_base.copy()
gdf_rhv_rdpt_simple2=gdf_rhv_rdpt_simple5.copy()
et.calculer_trafic_mmm(gdf_calcul, gdf_rhv_rdpt_simple2,lgn_rdpt,graph_filaire_123_vertex,mmm_simple,cle_mmm_rhv,dico_noeud)

In [ ]:
#EXPORT RESULTATS
gdf_calcul['id_cpt_exp']=gdf_calcul['id_cpt_2_sens'].fillna('NC')
gdf_calcul['id_cpt_exp']=gdf_calcul.apply(lambda x : ', '.join([str(a) for a in x['id_cpt_exp']]) 
                if isinstance(x['id_cpt_2_sens'],tuple) else str(x['id_cpt_2_sens']), axis=1)
gdf_calcul.reset_index()[['id', 'ident', 'domanial', 'groupe', 'cat_dig', 'cat_rhv', 'passage',
       'rggraph_nd', 'rggraph_na', 'rgraph_dbl', 'numero', 'cdate', 'mdate',
       'id_ign', 'nature', 'sens', 'codevoie_d', 'importance', 'id_y','source', 'target',
       'idtronc', 'geom','tmjo_2_sens','type_cpt', 'id_cpt_exp']].to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_123_calcul_mmm_v6.shp')

In [ ]:
# POUR TEST SUR UN NOEUD UNIQUEMNET METHODE 3 VOIES
gdf_base2=gdf_traf_tot.copy()
gdf_rhv_rdpt_simple3=gdf_rhv_rdpt_simple.copy()
#remplacement des sources et targets : 
simplifier_noeud_rdpt(gdf_rhv_rdpt_simple3, dico_noeud)
#puis recalculer le count du nb de ligne par noeud (en otant les lignes qui font les rdpoints)
df_noeuds,liste_noeud_estim=et.noeuds_estimables(gdf_rhv_rdpt_simple3)[2:4]
liste_noeud_traite=[]
liste_noeud_mmm=[]
dico_erreur={}
num_noeud=2082
gdf_rhv_rdpt_simple3=verif_index(gdf_rhv_rdpt_simple3,'idtronc')#Îsi idtronc est en index on le repasse dans les colonnes

df_troncon_noeud=et.carac_troncon_noeud(gdf_rhv_rdpt_simple3, gdf_base2, graph_filaire_123_vertex,num_noeud,df_noeuds,lgn_rdpt)
  #si tous les trafics du noeud sont déjà renseignés on passe
  #determiner si on est dans le cas d'une voie pouvant etre estimée par calcul (3 troncon dont 1 seul manquant) ou par analogie avec le MMM
et.maj_trafic_3tronc(df_troncon_noeud)
df_update_traf=df_troncon_noeud[['idtronc','tmjo_2_sens','type_cpt']].set_index('idtronc').drop_duplicates()
df_update_traf.update(pd.DataFrame.from_records(dico_corection_calcul_3_voies, columns=['idtronc','tmjo_2_sens', 'type_cpt']).set_index('idtronc').fillna('NC'))

In [ ]:
#TEST APPARtENANCE A UN ROND POINT
for a in [9314,6404,6403, 8206, 9315] : 
    if a in df_noeuds.noeud.tolist() : 
        print(a)

In [ ]:
# POUR TEST SUR UN NOEUD UNIQUEMNET METHODE MMM
gdf_test=gdf_base.copy()
df_noeuds,liste_noeud_estim=et.noeuds_estimables(gdf_rhv_rdpt_simple5)[2:4]
num_noeud=1002
dico_erreur={}
df_troncon_noeud=et.carac_troncon_noeud(gdf_rhv_rdpt_simple5, gdf_test, graph_filaire_123_vertex,num_noeud,df_noeuds,lgn_rdpt)
dico_troncons_noeud=et.creer_dico_troncons_noeud(df_troncon_noeud,gdf_rhv_rdpt_simple5,gdf_base,lgn_rdpt,graph_filaire_123_vertex )
matrice_rhv=et.matrice_troncon_noeud_rhv(df_troncon_noeud,num_noeud,lgn_rdpt,dico_troncons_noeud) 
try :
    joint_fv_mmm_e2=et.estim_mmm_jointure_voies(matrice_rhv,cle_mmm_rhv)
except et.PasCorrespondanceError : 
    dico_erreur[num_noeud]='PasCorrespondanceError'
    df_noeuds,liste_noeud_estim=et.noeuds_estimables(gdf_rhv_rdpt_simple5)[2:4]
    liste_noeud_estim=[n for n in liste_noeud_estim if n not in liste_noeud_traite]
try :
    trafic_inconnus_prior_cat=et.isoler_trafic_inconnu(joint_fv_mmm_e2)
except et.PasDeTraficInconnuError: 
    dico_erreur[num_noeud]='PasDeTraficInconnuError' 
    df_noeuds,liste_noeud_estim=et.noeuds_estimables(gdf_rhv_rdpt_simple5)[2:4]
    liste_noeud_estim=[n for n in liste_noeud_estim if n not in liste_noeud_traite]
try :
    trafc_rens=et.trafic_mmm(gdf_test,trafic_inconnus_prior_cat,mmm_simple,cle_mmm_rhv, df_troncon_noeud,dico_troncons_noeud,lgn_rdpt,dico_noeud,num_noeud)
    df_update_traf=et.calcul_trafic_rhv_depuisMMM(trafc_rens).set_index('idtronc').rename(columns={'tmjo_2_sens_extrapol':'tmjo_2_sens'})
except et.PasDeTraficError as e: 
    dico_erreur[num_noeud]=e 
    df_noeuds,liste_noeud_estim=et.noeuds_estimables(gdf_rhv_rdpt_simple5)[2:4]
    liste_noeud_estim=[n for n in liste_noeud_estim if n not in liste_noeud_traite]

# Voies catégorie 4

## Affectation comptages ponctuels
> Normalement les comptages ponctuels sont deja assignes aux idtronc relatifs au troncon elementaires toutes catégories. Il faut donc isoler les voies non contenues dans le regrouepement des cat 1,2,3

In [176]:
rhv_grp_hors_123,affect_finale_cat4,cpt_perm_cat4=v4.isolerDonneesCat4(gdf_rhv_groupe,gdf_rhv_groupe_123, affect_finale,lgn_proche_perm)
groupe_ident_cpt,cpt_cat4_uniq_sens_dbl,cat4_lgn_pb=v4.jointureTraficLigne(rhv_grp_hors_123,affect_finale_cat4,cpt_perm_cat4)
rhv_grp_hors_123_traf=v4.calculTraficPointComptage(rhv_grp_hors_123,groupe_ident_cpt,cpt_cat4_uniq_sens_dbl,cat4_lgn_pb,[3094, 4687, 8724, 10025])

In [ ]:
#fichier avec cat1,2,3 et comptage sur cat4
gdf_traf_1234=pd.concat([rhv_grp_hors_123_traf.loc[~rhv_grp_hors_123_traf.tmjo_2_sens.isna()],gdf_calcul], axis=0, sort=False)

In [ ]:
gdf_traf_1234.reset_index()[['id', 'ident', 'domanial', 'groupe', 'cat_dig', 'cat_rhv', 'passage',
       'rggraph_nd', 'rggraph_na', 'rgraph_dbl', 'numero', 'cdate', 'mdate',
       'id_ign', 'nature', 'sens', 'codevoie_d', 'importance', 'id_y','source', 'target',
       'idtronc', 'geom','tmjo_2_sens','type_cpt', 'id_cpt_exp']].to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_1234_cpt.shp')

## Affectation auto à partir du bati 
> L'idée c'est d'affecter un tmja a chaque troncon elementaire à partir du nb d'habitantassocier à la ligne la plus proche, puis de moduler ça selon le nb de déplcament motorisée lies au domicile. si apres on arrive aussi a ajouter les tmja pour les voies non reliees aux reseau de cat 1,2,3 c'est parfait

In [8]:
#import des donnees
gdf_traf_1234, voies_nc, bati,coreesp_bati2=v4.importDonneesBase(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_1234_cpt.shp',
                                                                 r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\voie_non_affectees.shp',
                                                                 r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\bati.shp',
                                                                 r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\ppv_bati-route_V2.csv')

### Recherche des trajets les plus courts vers une voie au trafic connu
> test sur un seul batiment  : 'BATIMENT0000000256067979'.<br> on va alimenter des listes de parcours : list des vertex à tester, list des ident parcouru,list des vertex parcouru

> **Test : creation d'une classe ensemble de trajet**<br>
>>doit permettre : 
* de verifier le statut de tous les trajets. method : verif_statut_trajets()
* de calculer tous les trajets. method : calcul_trajets()
* trouver les trajets non finis. method : trajets_en_cours()
* creer des nouveaux trajets. method : creer_trajet()
* trouver les lignes a ajouter aux trakets en cours. method : trouver_nouvelles_lignes()
* ajouter une ligne à un trajet existant. method : ajouter_ligne.<br> 

>>Se base sur : 
* lignes depart
* ensemble de lignes
* vertex connus
* vertex_impasse

In [7]:
#recuperer les donnees necessaire a la recherche du trajet le plus rapide vers un point non connu
bati_ligne_proche,ensemble_lignes,vertex_connus, vertex_impasse=v4.definitionVariables(graph_filaire,gdf_rhv_groupe,gdf_traf_1234, voies_nc, bati,coreesp_bati2)

In [8]:
%%time
#test sur une ligne
#il peut y avoir des erreurs dans le rhv, comme par exemple l'ident 14844 qui ne touche pas le 24285
ident_proche_bati=str(bati_ligne_proche.loc[bati_ligne_proche['ID']=='BATIMENT0000000257801129'].ident.to_numpy()[0])#risque si ident pas trouver d'IndexError
point_depart=bati_ligne_proche.loc[bati_ligne_proche['ID']=='BATIMENT0000000257801129'].point_proche.values[0]
trajets=v4.ensemble_trajet(ident_proche_bati,point_depart,ensemble_lignes, vertex_connus, vertex_impasse, graph_filaire_vertex)
trajets.initialiser_trajets() 
trajets.allonger_trajet()
trajets.creer_df_tt_trajet()
trajets.filtrer_df_tt_trajet()
trajets.trajet_court=tuple(trajets.df_trajet_cat3_grp_OD.loc[trajets.df_trajet_cat3_grp_OD.longueur_tot==trajets.df_trajet_cat3_grp_OD.longueur_tot.min()].lignes.to_numpy()[0])
trajets.point_arrive=trajets.df_trajet_cat3_grp_OD.loc[trajets.df_trajet_cat3_grp_OD.longueur_tot==trajets.df_trajet_cat3_grp_OD.longueur_tot.min()].points.to_numpy()[0][-1]
#RESULTATS (cf module et attributs)
trajets.trajet_court
#trajets.df_tt_trajet

Wall time: 10.7 s


('24279', '14826', '51783', '51782')

In [ ]:
#mise en oeuvre
#resultat=r_tot.copy()
resultat=pd.DataFrame({'bati':[], 'ident':[], 'noeud_proche':[],'trajet_court':[],'comm':[], 'noeud_cat3':[]})
#dico_df_tt_trajet=d_tot
dico_df_tt_trajet={}
#i=r_tot.index.max()+1
i=0
liste_bati_erreur=[]
for e,b in enumerate(bati_ligne_proche['ID'].tolist()) : 
    if e%100==0 and e!=0:
        print(f'nb bati traite : {e}, {datetime.now()},{b}')
    try : #recuperer l'ident pres du bati a traiter, si pas d'ident, on continue
        ident_proche_bati=str(bati_ligne_proche.loc[bati_ligne_proche['ID']==b].ident.to_numpy()[0])#risque si ident pas trouver d'IndexError
    except IndexError : 
        liste_bati_erreur.append(b)
        continue
    if ensemble_lignes.loc[ensemble_lignes['ident']==ident_proche_bati].empty : #si l'ident du rhv proche du bati n'est pas dans la liste, on indique et on continue
        resultat.loc[i]=[b,ident_proche_bati,np.NaN,np.NaN,'pas ident proche dans voies nc',np.NaN]
        i+=1
        continue
    point_depart=bati_ligne_proche.loc[bati_ligne_proche['ID']==b].point_proche.values[0]
    trajets=v4.ensemble_trajet(ident_proche_bati,point_depart,ensemble_lignes, vertex_connus, vertex_impasse, graph_filaire_vertex)
    noeud_proche=trajets.noeud_proche
    if ident_proche_bati in resultat.ident.tolist() : #si la ligne a deja ete traite avec un autre batiment
        df_find_existant=resultat.loc[(resultat['ident']==ident_proche_bati) & (resultat['noeud_proche']==noeud_proche)] # si le noeud le plus proche est le mm que celui dun bati precedent, on ne fait que recuperer les donnees calculees car on est dans le mm cas
        if not df_find_existant.empty:
            resultat.loc[i]=[b,ident_proche_bati,noeud_proche,df_find_existant.trajet_court.values[0], 'Deduction_trajet-ident connu_noeud_proche connu',df_find_existant.noeud_cat3.values[0]]
            i+=1
        else : #sinon on deroule le processus
            trajets.df_trajet_cat3_grp_OD=dico_df_tt_trajet[ident_proche_bati]
            trajets.maj_longueur_tot(trajets.df_trajet_cat3_grp_OD)
            df_finale=trajets.df_trajet_cat3_grp_OD.loc[trajets.df_trajet_cat3_grp_OD.longueur_tot==trajets.df_trajet_cat3_grp_OD.longueur_tot.min()]
            resultat.loc[i]=[b,ident_proche_bati,noeud_proche,tuple(df_finale.lignes.to_numpy()[0]), 'Deduction_trajet-ident connu_noeud_proche varie',df_finale.points.to_numpy()[0][-1]]
            i+=1
    elif ident_proche_bati in [b for a in resultat.loc[~resultat['trajet_court'].isna()].trajet_court.to_numpy() for b in a] : #si l'ident est sur un chemin deja traite
        resultat_equi=resultat.loc[resultat.trajet_court.apply(lambda x : ident_proche_bati in x if isinstance(x,tuple) else False)].iloc[0]
        trajet_equi=resultat_equi.trajet_court
        trajet_court=trajet_equi[trajet_equi.index(ident_proche_bati):] 
        resultat.loc[i]=[b,ident_proche_bati,trajets.src,trajet_court, 'Deduction_trajet-ident sur chemin',resultat_equi.noeud_cat3]
        i+=1
        resultat.loc[i]=[b,ident_proche_bati,trajets.tgt,trajet_court, 'Deduction_trajet-ident sur chemin',resultat_equi.noeud_cat3]
        i+=1
    else : #sinon on deroule le calcul normalement
        try :
            trajets.calculs_trajets()
        except v4.PasDeCheminCat3Error : 
            liste_bati_erreur.append(b)
            continue
        resultat.loc[i]=[b,ident_proche_bati,noeud_proche,trajets.trajet_court, 'Calcul_trajet',trajets.point_arrive]
        i+=1
        dico_df_tt_trajet[ident_proche_bati]=trajets.df_trajet_cat3_grp_OD

In [ ]:
#en cas d'arret pour erreur
#reste_a_faire=export.iloc[:40000].loc[export.iloc[:40000].ID.isin(resultat.bati.tolist())].copy()
index_arret=list_ids.index(resultat.tail(1).bati.values[0])+1
index_bati_erreur=list_ids[index_arret]

In [16]:
#SAVE RESULTATS FINAUX
resultat.to_json(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\resultats_V2.json')

dico_save2={k:v.to_dict() for k, v in dico_df_tt_trajet.items()}
with open(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\df_tt_trajet_resultats_V2.json','w') as f:
    f.write(json.dumps(dico_save2))

In [ ]:
#rappel
with open(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\df_tt_trajet_resultats_V2.json','r') :
    retour_dict=json.loads(f.read())
dico_retour={k:pd.DataFrame.from_dict(v) for k, v in retour_dict.items()}
r_tot=pd.read_json(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\resultats_V2.json')
#donnees erreur
with open(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\liste_bati_erreur.json','w') as f :
    f.write(json.dumps(liste_bati_erreur))

In [9]:
resultats_finaux=pd.read_json(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\resultats_V2.json')
#ancienne version : r'D:\Boulot\Lin_bm\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\resultats_tot.json

In [21]:
#ramener les trajets par batiment a une association 1..1 pour le calcul de la somme
resultats_finaux_ordonnees=resultats_finaux.explode('trajet_court')
resultats_finaux_ordonnees.reset_index(drop=True).to_json(r'D:\Boulot\Lin_bm\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\resultats_tot_ordonnes_V2.json')

In [10]:
#si besoin recup des donnees
resultats_finaux_ordonnees=pd.read_json(r'D:\Boulot\Lin_bm\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\resultats_tot_ordonnes.json')

#jointure avec bati pour recuperer nb personnes et geom
resultats_finaux_ordonnees_pop=bati[['ID','PopT2016','geometry']].merge(resultats_finaux_ordonnees,left_on='ID', right_on='bati')

#jointire spatiale avec EMC selon secteur pour connaitre le nombre total de deplacement par ident du rhv
zones_emc=gp.read_file(r'D:\Boulot\Lin_bm\C19SA0101\Doc_travail\Donnees_produites\Donnees\zones\zonage_eda2017.shp')

emc_nbpers_rhv=gp.sjoin(resultats_finaux_ordonnees_pop,zones_emc,how='left',op='intersects')

#il y a qq batiment duplique, mais a cause de la cission des bati par le lCSQA, la prochaine fois prendre attribut IDBATI
#emc_nbpers_rhv.loc[emc_nbpers_rhv.duplicated(['bati','trajet_court','PopT2016'], keep=False)].sort_values(['bati','trajet_court'])
#recuperer le nombre de trajet moyen par secteur : 
trajMoyen_Emc=pd.read_csv(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\EMC\Linéarisation BdxM et EMC²\trajet_moy_redress_coeq.csv')
#affecter a chaque ident du rhv
trajMoyParLigneRhv=emc_nbpers_rhv.merge(trajMoyen_Emc[['SEC','nb_traj_moy']], left_on='num_zone', right_on='SEC')
#calcul du nombre de deplacements relatif a chaque batiment pour chaque identifiant
trajMoyParLigneRhv['nbDeplacement']=trajMoyParLigneRhv['PopT2016']*trajMoyParLigneRhv['nb_traj_moy']
#estimation du trafic par identifiant du rhv
TraficMoyenEstime=trajMoyParLigneRhv.groupby('ident').agg({'nbDeplacement':sum,'PopT2016':sum,'nb_traj_moy':np.mean}).reset_index()

In [25]:
TraficMoyenEstime.sort_values('nbDeplacement')

,ident,nbDeplacement,PopT2016,nb_traj_moy
3222,7333,0.423666,1.339451,0.316298
10770,22470,0.533505,1.686716,0.316298
15426,38974,0.678428,2.144902,0.316298
4659,10187,0.732721,2.316553,0.316298
3619,8097,0.756230,2.390878,0.316298
...,...,...,...,...
10131,21179,3916.350382,1974.187895,1.983778
11012,22962,4214.265872,2683.539505,1.570413
2400,5430,6568.206630,4369.978826,1.503029
2815,6509,7620.325399,9314.562460,0.818109


In [15]:
liste_bati_erreur

[]

In [19]:
resultat

,bati,ident,noeud_proche,trajet_court,comm,noeud_cat3
0,BATIMENT0000000257005411,30604,26022.0,"(30604, 44711, 44712)",Calcul_trajet,10489.0
1,BATIMENT0000000257005409,30604,26022.0,"(30604, 44711, 44712)",Deduction_trajet-ident connu_noeud_proche connu,10489.0
2,BATIMENT0000000257005395,30604,26022.0,"(30604, 44711, 44712)",Deduction_trajet-ident connu_noeud_proche connu,10489.0
3,BATIMENT0000000257005393,30604,26022.0,"(30604, 44711, 44712)",Deduction_trajet-ident connu_noeud_proche connu,10489.0
4,BATIMENT0000000257005408,30604,26022.0,"(30604, 44711, 44712)",Deduction_trajet-ident connu_noeud_proche connu,10489.0
...,...,...,...,...,...,...
148000,BATIMENT0000000248395177,10685,16524.0,"(10685, 10684, 32419)",Deduction_trajet-ident sur chemin,17414.0
148001,BATIMENT0000000248395177,10685,12273.0,"(10685, 10684, 32419)",Deduction_trajet-ident sur chemin,17414.0
148002,BATIMENT0000000254021494,13633,1611.0,"(13633, 13952, 13661, 13660)",Calcul_trajet,5714.0
148003,BATIMENT0000000256984921,4441,20378.0,"(4441,)",Deduction_trajet-ident sur chemin,3510.0


In [22]:
resultats_finaux_ordonnees

,bati,ident,noeud_proche,trajet_court,comm,noeud_cat3
0,BATIMENT0000000257005411,30604,26022.0,30604,Calcul_trajet,10489.0
0,BATIMENT0000000257005411,30604,26022.0,44711,Calcul_trajet,10489.0
0,BATIMENT0000000257005411,30604,26022.0,44712,Calcul_trajet,10489.0
1,BATIMENT0000000257005409,30604,26022.0,30604,Deduction_trajet-ident connu_noeud_proche connu,10489.0
1,BATIMENT0000000257005409,30604,26022.0,44711,Deduction_trajet-ident connu_noeud_proche connu,10489.0
...,...,...,...,...,...,...
148002,BATIMENT0000000254021494,13633,1611.0,13952,Calcul_trajet,5714.0
148002,BATIMENT0000000254021494,13633,1611.0,13661,Calcul_trajet,5714.0
148002,BATIMENT0000000254021494,13633,1611.0,13660,Calcul_trajet,5714.0
148003,BATIMENT0000000256984921,4441,20378.0,4441,Deduction_trajet-ident sur chemin,3510.0
